In [1]:
import numpy as np
import matplotlib.pyplot as plt
#from scipy.special import softmax
import note_seq
from note_seq.protobuf import music_pb2
import pandas as pd

%matplotlib inline

In [2]:
###################################
# MIDI // Audio utility functions
###################################

def create_note_sequence(notes, tempo=.1):
    ns = music_pb2.NoteSequence()
    t = 0
    for note in notes:
        ns.notes.add(pitch=60+note, start_time=t, end_time=t+tempo, velocity=80)
        t += tempo
    ns.total_time = t
    # ns.tempos.add(qpm=60)
    return ns

def write_midi_file(notes, outpath):
    ns = create_note_sequence(notes)
    note_seq.sequence_proto_to_midi_file(ns,outpath)
    return

# note_seq.plot_sequence(ns)
# note_seq.play_sequence(ns) fails in default version because of pretty_midi bug with Python3

In [3]:
######################################
# Tonal consonance utility functions
######################################

# h/r = sqrt(2/11)
diss_dict_chew_2_11 = {0: 1.0,
 1: 2.5584085962673253,
 2: 2.174229226018436,
 3: 1.9069251784911847,
 4: 1.7056057308448835,
 5: 1.4770978917519928,
 6: 3.247376563543955}

def is_empty_pc_hist(pc_hist):
    pc_hist = np.nan_to_num(pc_hist)
    for i in range(len(pc_hist)):
        if pc_hist[i] > 0: return False
    return True

def get_interval(pitch1, pitch2):
    interval = abs(pitch1-pitch2)
    if interval > 6:
        interval = 6-(interval-6)
    return interval

def ic_hist_to_avg_distance(ic_hist, topology=diss_dict_chew_2_11):
    ic_hist = ic_hist/np.sum(ic_hist) # normalize histogram

    distance = 0
    for i in range(7):
        distance += ic_hist[i]*topology[i]
    return distance

def pitch_to_interval_hist(pitch_histogram):
    intervals = [0]*7
    # count intervals
    for i in range(12):
        for j in range(i, 12):
            interval = get_interval(i, j)
            intervals[interval] += pitch_histogram[i]*pitch_histogram[j] # ToDo: is this the right way to go?
    return intervals

def get_cons_btw(pc_hist1, pc_hist2):
    intervals = [0]*7
    for i in range(12):
        for j in range(12):
            interval = get_interval(i, j)
            intervals[interval] += pc_hist1[i]*pc_hist2[j]
    return -ic_hist_to_avg_distance(intervals)

def get_consonance(pc_hist, topology=diss_dict_chew_2_11):
    if is_empty_pc_hist(pc_hist): return None
    ic_hist = pitch_to_interval_hist(pc_hist)
    return -ic_hist_to_avg_distance(ic_hist, topology)


def note_list_to_pc_hist(notes):
    pc_hist = [0]*12
    notes = [int(n) for n in notes]
    for note in notes:
        if note < 0: continue
        pc_hist[note] += 1
    return pc_hist

def combined_consonance(notesA, notesB, window=5, outpath=None):
    if (len(notesA) != len(notesB)): return False
    length = len(notesA)
    cons_vals = {'t': range(length), 'notesA': notesA, 'notesB': notesB,
                 'cc': [np.nan]*length, 'ec': [np.nan]*length,
                 'cA': [np.nan]*length, 'cB': [np.nan]*length}
    
    STEP = 1
    for i in range(window,len(notesA),STEP):
        pc_hist_A = note_list_to_pc_hist(notesA[i-window:i])
        pc_hist_B = note_list_to_pc_hist(notesB[i-window:i])
        pc_hist = pc_hist_A + pc_hist_B
        cons_vals['cc'][i] = get_consonance(pc_hist)
        cons_vals['cA'][i] = get_consonance(pc_hist_A)
        cons_vals['cB'][i] = get_consonance(pc_hist_B)
        cons_vals['ec'][i] = cons_vals['cc'][i] - np.nanmean((cons_vals['cA'],cons_vals['cB']))
    cons_df = pd.DataFrame(cons_vals)
    cons_df['window'] = window
    if outpath: cons_df.to_csv(outpath,index=False)
    return cons_df

def individual_consonance(notes, window=5, outpath=None):
    length = len(notes)
    cons_vals = {'t': range(length),'notes': notes,
                 'cA': [np.nan]*length}
    STEP = 1
    for i in range(window,length,STEP):
        pc_hist = note_list_to_pc_hist(notes[i-window:i])
        cons_vals['cA'][i] = get_consonance(pc_hist)
    cons_df = pd.DataFrame(cons_vals)
    if outpath: cons_df.to_csv(outpath,index=False)
    cons_df['window'] = window
    return cons_df

def individual_consonance_set(notes):
    pc_hist = note_list_to_pc_hist(notes)
    return get_consonance(pc_hist)

def combined_consonance_set(notes):
    notes = np.array(notes).flatten()
    return individual_consonance_set(notes)



# Gapped Consonance (between pitch sets) -- from normalized-gapped-consonance.ipynb
def get_cons_btw(pc_hist1, pc_hist2):
    intervals = [0]*7
    for i in range(12):
        for j in range(12):
            interval = get_interval(i, j)
            intervals[interval] += pc_hist1[i]*pc_hist2[j]
    return -ic_hist_to_avg_distance(intervals)

def normalized_gapped_cons(pc_hist1, pc_hist2):
    diss_within1 = get_consonance(pc_hist1)
    diss_within2 = get_consonance(pc_hist2)
    diss_btw = get_cons_btw(pc_hist1, pc_hist2)
    return -diss_btw/np.mean((diss_within1, diss_within2))

## Lagged Consonance Implementation

In [4]:
def get_combined_consonance(notes_df, window=5):
    cc = [None]*len(notes_df)
    window = 5
    for i in range(window-1,(1+len(notes_df))):
        notes = notes_df['notesA'].iloc[(i-window):i].values
        notes = np.concatenate((notes,notes_df['notesB'].iloc[(i-window):i].values))
        cc[i-1] = individual_consonance_set(notes)
    return cc

def lagged_consonance(notes_df, lag, window):
    lagged_cons = pd.DataFrame()

    # compute individual consonance time series
    lagged_cons['cA'] = notes_df['notesA'].rolling(window).apply(individual_consonance_set,raw=True)
    lagged_cons['cB'] = notes_df['notesB'].rolling(window).apply(individual_consonance_set,raw=True).shift(lag)

    # shift notesB by lag and get CC over rolling window
    notesB = notes_df['notesB'].shift(lag).fillna(-1).astype('int')
    notesAB = np.array([[notes_df['notesA'].iloc[i],notesB.iloc[i]] for i in range(len(notesB))]).flatten()
    lagged_cons['cc'] = pd.Series(notesAB).rolling(window*2).apply(individual_consonance_set,raw=True).iloc[1::2] # (n+3.247)/2.247 to normalize, see R code
    
    # get EC over rolling window
    cAB = lagged_cons.loc[:,['cA','cB']].mean(1)
    lagged_cons['ec'] = lagged_cons['cc'] - cAB
    # added this line for debugging
    lagged_cons['ec_norm'] = (lagged_cons['cc']+3.247)/2.247 - (cAB+3.247)/2.247
    lagged_cons['lag'] = lag
    lagged_cons['window'] = window
    lagged_cons['t'] = range(len(lagged_cons))
    
    return lagged_cons.dropna()

In [11]:
from glob import glob

lags = [int(i) for i in np.linspace(-20,20,41)]
files = glob('output-max-cons-time-decay/notes/*tau0.1-b5-*csv')
for f in files:
    print(f)
    #df = pd.read_csv(f)
    lagged_df = pd.DataFrame()
    for lag in lags:
        print(lag)
        df = pd.read_csv(f)
        lagged_df = pd.concat((lagged_df,lagged_consonance(df,lag,5)))
    lagged_df.to_csv('output-max-cons-time-decay/lagged-consonance/'+f.split('/')[-1],index=False)

output-max-cons-time-decay/notes/coupled-unseeded-tau0.1-b5-trial2.csv
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
output-max-cons-time-decay/notes/oneway-unseeded-tau0.1-b5-trial1.csv
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
output-max-cons-time-decay/notes/coupled-unseeded-tau0.1-b5-trial1.csv
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
output-max-cons-time-decay/notes/oneway-unseeded-tau0.1-b5-trial0.csv
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
output-max-cons-time-decay/notes/coupled-unseeded-tau0.1-b5-trial0.csv
-20
-19
-18
-17
-16
-15
-14
-13
-12
-11
-10
-9
-8
-7
-6
-5
-4
-3
-2
-1
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
output-max-cons-time-decay/

In [18]:
!mkdir output-max-cons-time-decay/lagged-consonance

## Plot lagged-consonance 